In [3]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import numpy as np
import copy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import clean as clean

In [5]:
# Load the raw data
dt_raw = pd.read_csv('../data/TBI PUD 10-08-2013.csv')
dt_clean = clean.clean_data('../data/TBI PUD 10-08-2013.csv', remove_feats_after_ct=True, remove_TBI_rows_with_nan=True,threshold=0.5,rm_feats=True,remove_GCS_total_mismatch=True)

In [6]:
# Split into training and test sets
train_df, test_df = train_test_split(dt_clean, test_size=0.2, random_state=42)

# Separate features (X) and target (y)
y_train = train_df["PosIntFinal"]
X_train = train_df.drop(columns=["PosIntFinal"])
y_test = test_df["PosIntFinal"]
X_test = test_df.drop(columns=["PosIntFinal"])

In [7]:
# Impute missing values in X_train and X_test
imputer = SimpleImputer(strategy='mean')  # Impute with mean values
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Initialize models
qda_model = QuadraticDiscriminantAnalysis()

In [8]:
# Train QDA model
qda_model.fit(X_train, y_train)

# Predictions for QDA
y_pred_qda = qda_model.predict(X_test)
y_proba_qda = qda_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
# save the model in results folder
np.save('../results/qda.npy', y_proba_qda)

/Users/vidhisinha/Downloads/ENTER/envs/215a/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:947: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [9]:
# Evaluation Metrics for QDA
print("\n=== QDA Evaluation ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_qda))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_qda))
print("\nROC AUC Score:")
print(roc_auc_score(y_test, y_proba_qda))


=== QDA Evaluation ===
Confusion Matrix:
[[7713  541]
 [  49  105]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96      8254
         1.0       0.16      0.68      0.26       154

    accuracy                           0.93      8408
   macro avg       0.58      0.81      0.61      8408
weighted avg       0.98      0.93      0.95      8408


ROC AUC Score:
0.9194632905258057
